All 3 channels (rgb) of .jpg are identical

In [ ]:
import os
import nibabel as nib
import matplotlib.pyplot as plt
from glob import glob

def extract_slices_from_folder(input_folder, output_base_dir):
    # Get all .nii.gz files in the input folder
    nii_files = glob(os.path.join(input_folder, '*.nii.gz'))

    for nii_file in nii_files:
        # Load the NIfTI file
        img = nib.load(nii_file)
        data = img.get_fdata()

        # Get the filename without extension
        base_name = os.path.splitext(os.path.splitext(os.path.basename(nii_file))[0])[0]

        # Create output folders
        folders = [f"{base_name}_x", f"{base_name}_y", f"{base_name}_z"]
        for folder in folders:
            os.makedirs(os.path.join(output_base_dir, folder), exist_ok=True)

        # Extract and save slices
        for axis in range(3):
            for i in range(data.shape[axis]):
                if axis == 0:
                    slice_data = data[i, :, :]
                    out_file = os.path.join(output_base_dir, f"{base_name}_x", f"{base_name}_x_{i:03d}.jpg")
                elif axis == 1:
                    slice_data = data[:, i, :]
                    out_file = os.path.join(output_base_dir, f"{base_name}_y", f"{base_name}_y_{i:03d}.jpg")
                else:
                    slice_data = data[:, :, i]
                    out_file = os.path.join(output_base_dir, f"{base_name}_z", f"{base_name}_z_{i:03d}.jpg")

                plt.imsave(out_file, slice_data, cmap='gray')

        print(f"Slices extracted from {base_name} and saved in folders: {', '.join(folders)}")

    print("All .nii.gz files processed.")

# Usage example
input_folder = r"C:\Users\acer\Desktop\Data_0\Nii_cropped_224"
output_base_dir = r"C:\Users\acer\Desktop\Data_0\Nii_toJPG"
extract_slices_from_folder(input_folder, output_base_dir)

3 channels of .jpg contain information from layers n-1 n and n+1

In [ ]:
import os
import nibabel as nib
import numpy as np
from PIL import Image
from glob import glob

def extract_slices_from_folder(input_folder, output_base_dir):
    nii_files = glob(os.path.join(input_folder, '*.nii.gz'))

    for nii_file in nii_files:
        img = nib.load(nii_file)
        data = img.get_fdata()

        base_name = os.path.splitext(os.path.splitext(os.path.basename(nii_file))[0])[0]

        folders = [f"{base_name}_x", f"{base_name}_y", f"{base_name}_z"]
        for folder in folders:
            os.makedirs(os.path.join(output_base_dir, folder), exist_ok=True)

        for axis in range(3):
            for i in range(1, data.shape[axis] - 1):  # Start from 1 and end at shape-2
                if axis == 0:
                    slice_data = np.stack([
                        data[i-1, :, :],
                        data[i, :, :],
                        data[i+1, :, :]
                    ], axis=-1)
                    out_file = os.path.join(output_base_dir, f"{base_name}_x", f"{base_name}_x_{i:03d}.jpg")
                elif axis == 1:
                    slice_data = np.stack([
                        data[:, i-1, :],
                        data[:, i, :],
                        data[:, i+1, :]
                    ], axis=-1)
                    out_file = os.path.join(output_base_dir, f"{base_name}_y", f"{base_name}_y_{i:03d}.jpg")
                else:
                    slice_data = np.stack([
                        data[:, :, i-1],
                        data[:, :, i],
                        data[:, :, i+1]
                    ], axis=-1)
                    out_file = os.path.join(output_base_dir, f"{base_name}_z", f"{base_name}_z_{i:03d}.jpg")

                # Normalize the data to 0-255 range
                slice_data = ((slice_data - slice_data.min()) / (slice_data.max() - slice_data.min()) * 255).astype(np.uint8)
                
                # Create and save the image
                img = Image.fromarray(slice_data)
                img.save(out_file)

        print(f"Slices extracted from {base_name} and saved in folders: {', '.join(folders)}")

    print("All .nii.gz files processed.")

# Usage example
input_folder = r"C:\Users\acer\Desktop\Data_0\Nii_cropped_224"
output_base_dir = r"C:\Users\acer\Desktop\Data_0\Nii_toJPG"
extract_slices_from_folder(input_folder, output_base_dir)

In [ ]:
import os
import nibabel as nib
import numpy as np
from PIL import Image
from glob import glob

def extract_slices_from_folder(input_folder, output_base_dir):
    nii_files = glob(os.path.join(input_folder, '*.nii.gz'))

    for nii_file in nii_files:
        img = nib.load(nii_file)
        data = img.get_fdata()

        base_name = os.path.splitext(os.path.splitext(os.path.basename(nii_file))[0])[0]

        folders = [f"{base_name}_x", f"{base_name}_y", f"{base_name}_z"]
        for folder in folders:
            os.makedirs(os.path.join(output_base_dir, folder), exist_ok=True)

        for axis in range(3):
            for i in range(224):  # Always generate 224 images

                if axis == 0:
                    slice_data = np.zeros((data.shape[1], data.shape[2], 3), dtype=np.float32)
                    if i > 0:
                        slice_data[:, :, 0] = data[i-1, :, :] if i-1 < data.shape[0] else 0
                    slice_data[:, :, 1] = data[i, :, :] if i < data.shape[0] else 0
                    if i < data.shape[0] - 1:
                        slice_data[:, :, 2] = data[i+1, :, :] if i+1 < data.shape[0] else 0
                    out_file = os.path.join(output_base_dir, f"{base_name}_x", f"{base_name}_x_{i:03d}.jpg")

                elif axis == 1:
                    slice_data = np.zeros((data.shape[0], data.shape[2], 3), dtype=np.float32)
                    if i > 0:
                        slice_data[:, :, 0] = data[:, i-1, :] if i-1 < data.shape[1] else 0
                    slice_data[:, :, 1] = data[:, i, :] if i < data.shape[1] else 0
                    if i < data.shape[1] - 1:
                        slice_data[:, :, 2] = data[:, i+1, :] if i+1 < data.shape[1] else 0
                    out_file = os.path.join(output_base_dir, f"{base_name}_y", f"{base_name}_y_{i:03d}.jpg")
                    
                else:
                    slice_data = np.zeros((data.shape[0], data.shape[1], 3), dtype=np.float32)
                    if i > 0:
                        slice_data[:, :, 0] = data[:, :, i-1] if i-1 < data.shape[2] else 0
                    slice_data[:, :, 1] = data[:, :, i] if i < data.shape[2] else 0
                    if i < data.shape[2] - 1:
                        slice_data[:, :, 2] = data[:, :, i+1] if i+1 < data.shape[2] else 0
                    out_file = os.path.join(output_base_dir, f"{base_name}_z", f"{base_name}_z_{i:03d}.jpg")

                # Normalize the data to 0-255 range
                slice_data = ((slice_data - slice_data.min()) / (slice_data.max() - slice_data.min() + 1e-8) * 255).astype(np.uint8)
                
                # Create and save the image
                img = Image.fromarray(slice_data)
                img.save(out_file)

        print(f"224 slices extracted from {base_name} and saved in folders: {', '.join(folders)}")

    print("All .nii.gz files processed.")

# Usage example
input_folder = r"C:\Users\acer\Desktop\Data_0\Nii_cropped_224"
output_base_dir = r"C:\Users\acer\Desktop\Data_0\Nii_toJPG"
extract_slices_from_folder(input_folder, output_base_dir)